<a href="https://colab.research.google.com/github/sufianj/fast-camembert/blob/main/Fast_BERT_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fast-bert

     |████████████████████████████████| 92kB 8.2MB/s 
     |████████████████████████████████| 3.0MB 28.1MB/s 
     |████████████████████████████████| 317kB 52.0MB/s 
     |████████████████████████████████| 778kB 54.7MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 1.2MB 51.1MB/s 
     |████████████████████████████████| 890kB 52.6MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=2424aae508e04d67ede8bcb88ea2cb226057735f58d5f240a63ca01361cff2a6
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c401d0a570f812da19cc93dfc3f4a5380a11d2fc05fc32cc5373b778e5591f67
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses


In [ ]:
!rm -rf apex/

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
% cd apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 7910, done.
remote: Total 7910 (delta 0), reused 0 (delta 0), pack-reused 7910
Receiving objects: 100% (7910/7910), 13.99 MiB | 28.32 MiB/s, done.
Resolving deltas: 100% (5396/5396), done.
setup.sh: 4: setup.sh: %: not found
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-uvkaubwl
Created temporary directory: /tmp/pip-req-tracker-axl7a9vn
Created requirements tracker '/tmp/pip-req-tracker-axl7a9vn'
Created temporary directory: /tmp/pip-install-xsaiuh7_
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-rrr0tyw7
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-axl7a9vn'
    Running setup.py (path:/tmp/pip-req-build-rrr0tyw7/setup.py) egg_info for package 

In [ ]:
import torch
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging
import datetime
import sys

device_cuda = torch.device("cuda")

In [ ]:
DATA_PATH = Path('/content/drive/MyDrive/fast-bert/data/')
LOG_PATH = Path('/content/drive/MyDrive/fast-bert/logs') 
MODEL_PATH = Path('/content/drive/MyDrive/fast-bert/model/')
LABEL_PATH = Path('/content/drive/MyDrive/fast-bert/labels/')

In [ ]:
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, "web multi-class classification with bert"))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])
logger = logging.getLogger()

In [ ]:
#let's just check the training set and use it to do the fine-tuning
df_train = pd.read_csv("/content/drive/MyDrive/fast-bert/data/train_set.csv")

### **Création de LMDataBunch**

In [ ]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='bert-base-uncased',
                    batch_size_per_gpu=8, #was 16, even 8 won't do
                    max_seq_length=512, #was 512
                    multi_gpu=True,
                    model_type='bert',
                    logger=logger)

In [ ]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='bert-base-uncased',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=True,
                            logging_steps=500,
                            is_fp16=True) #was true with gpu

In [ ]:
lm_learner.fit(epochs=3, #was 30
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

In [ ]:
lm_learner.validate()

In [ ]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='clean_text',
                          label_col=labels,
                          batch_size_per_gpu=8,
                          max_seq_length=512,
                          multi_gpu=True,
                          multi_label=True,
                          model_type='bert')

In [ ]:
lm_learner.save_model()

In [ ]:
# issue fast-bert pos_weight <= downgrade to 1.9.1 solve the prob
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='/content/drive/MyDrive/fast-bert/model/model_out',
                        metrics=metrics,
                        device=device_cuda, #was device_cuda
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=True,
                        multi_label=True,
                        is_fp16=True,#True when is cuda
                        logging_steps=8000)#was 50, aïe, #todo: fix tf board

In [ ]:
cl_learner.fit(epochs=10,# was 30
            lr=1e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

In [ ]:
cl_learner.validate()

In [ ]:
cl_learner.save_model()

In [ ]:
predictor = BertClassificationPredictor(
                model_path='/content/drive/MyDrive/fast-bert/finetuned_model/model_out',
                label_path='/content/drive/MyDrive/fast-bert/labels/',
                multi_label=True,
                model_type='bert',
                do_lower_case=False)

In [ ]:
#evaluation on fine-tuning set 
df_val = pd.read_csv('/content/drive/MyDrive/fast-bert/data/val_set.csv')

#predictor.get_learner()

In [ ]:
batch_predictions = predictor.predict_batch(df_val.clean_text.to_list())

In [ ]:
batch_predictions[:10]